# CppImport magic's documentation
`cppimport.magic` is an [IPython](http://ipython.org) extension that help to use C/C++ code in an interactive session.

* Author `cppimport`:  T. Ben Thompson (t.ben.thompson@gmail.com)
* `cppimport.magic`: Serguei E. Leontiev (leo@sai.msu.ru)
* Homepage: https://github.com/tbenthompson/cppimport
* SPDX-License-Identifier: MIT

### TODO
- Editorial edits are required. Sorry for my best Engish;
- Now, don't have `boost`, `numpy`, `blas` and `lapack` for Python 3.12.0b4 (cells tagged as `xfail_python_3_12`)

## Install or upgrade
You can install or upgrade via `pip`:
```
        pip install -U cppimport
```

## Usage
Then you are ready to load the magic:

In [1]:
%load_ext cppimport.magic

To load it each time IPython starts, list it in your configuration file:
```
    c.InteractiveShellApp.extensions = [
        'cppimport.magic'
    ]
```

In [2]:
%%cppimport --help #
#

::

  %cppimport [-v] [--help] cpp_module

Build and import C/C++ module from ``%%ccpimport`` cell

The content of the cell is written to a file in the
directory ``$IPYTHONDIR/cppimport/<random>/<hash>/`` using
a dirname with the hash of the code, flags and configuration
data. This file is then compiled. The resulting module is
imported.

Usage
=====
Prepend ``%%cppimport`` to your C++/C code in a cell:

%%cppimport module.cpp
// put your code here.

positional arguments:
  cpp_module       Module C/C++ source file name.

options:
  -v, --verbosity  Increase output verbosity.
  --help           Print docstring as output cell.



In [3]:
%cppimport_config --help

::

  %cppimport_config [-v] [--clean-cache] [--defaults] [--help]

options:
  -v, --verbosity  Increase output verbosity.
  --clean-cache    Clean ``cppimport.magic`` build cache.
  --defaults       Delete custom configuration and back to default.
  --help           Print docstring as output cell.



## Verbosity
By default, magic returns output data only if errors or warnings occur during compilation. But you can increase the verbosity with the flag ``-v``:

Flags    | Description
:--------|:-----------
None     | No outputs if successful
``-v``   | List top level objects of imporing module
``-vv``  | Setuptools verbose output
``-vvv`` | Debug logging

In [4]:
%cppimport_config -v

INF:cppimport.magic:New default arguments for %%cppimport: -v


In [5]:
%cppimport_config --clean-cache
first_replay_test = len(In)

INF:cppimport.magic:Clean cache: /Users/leo/.cache/ipython/cppimport/aa1e4ca1


## C/C++ code cell's
You can use any interface libraries: [py11bind](https://pybind11.readthedocs.io), [Boost.Python](https://www.boost.org/doc/libs/release/libs/python/), native [Python/C API](https://docs.python.org/3/c-api/index.html) etc.

Additional build flags configure by `cfg` dictionary in [Mako](https://www.makotemplates.org/) code blocks (`<%` and `%>`).

### Use py11bind
`setup_pybind11(cfg)` - short configuration definition for the comparatively popular C++11/Python interface.

Also, classes defined by the cell (module) should be annotated as local (`py::module_local`). Classes without such an annotation will conflict with themselves when recompiled. See example [Use header files](#use_header_files) below.

In [6]:
%%cppimport somecode.cpp
// Source:
// https://github.com/tbenthompson/cppimport
#include <pybind11/pybind11.h>

namespace py = pybind11;

int square(int x) {
    return x * x;
}

PYBIND11_MODULE(somecode, m) {
    m.def("square", &square);
}
/*<%
setup_pybind11(cfg)
%>*/

INF:cppimport.magic:C/C++ objects: somecode.square


In [7]:
somecode.square(12)

144

In [8]:
assert 4 == somecode.square(2)

#### C++14 py11bind example
To use [py11bind](https://pybind11.readthedocs.io) with C++14/17/23/... code, you must explicitly set the appropriate compiler flag.

In [9]:
%%cppimport cpp14module.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/cpp14module.cpp
#include <pybind11/pybind11.h>

namespace py = pybind11;

// Use auto instead of int to check C++14
auto add(int i, int j) {
    return i + j;
}

PYBIND11_MODULE(cpp14module, m) {
    m.def("add", &add);
}
/*<%
setup_pybind11(cfg)
cfg['compiler_args'] += ['-std=c++14']
%>*/

INF:cppimport.magic:C/C++ objects: cpp14module.add


In [10]:
assert 18 == cpp14module.add(7, 11)

### Use C/Python API

In [11]:
%%cppimport raw_extension.c
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/raw_extension.c
#include <Python.h>

#if PY_MAJOR_VERSION >= 3
    #define MOD_INIT(name) PyMODINIT_FUNC PyInit_##name(void)
    #define MOD_DEF(ob, name, doc, methods) \
        static struct PyModuleDef moduledef = { \
            PyModuleDef_HEAD_INIT, name, doc, -1, methods, }; \
        ob = PyModule_Create(&moduledef);
    #define MOD_SUCCESS_VAL(val) val
#else
    #define MOD_INIT(name) PyMODINIT_FUNC init##name(void)
    #define MOD_DEF(ob, name, doc, methods) \
        ob = Py_InitModule3(name, methods, doc);
    #define MOD_SUCCESS_VAL(val)
#endif

static PyObject* add(PyObject* self, PyObject* args) {
    int a, b;
    //int class = 1;
    if (!PyArg_ParseTuple(args, "ii", &a, &b)) {
        return NULL;
    }
    return Py_BuildValue("i", a + b);
}

static PyMethodDef methods[] = {
    {"add", add, METH_VARARGS,
     "add(a: np.int32, b: np.int32) -> np.int32)"},
    {NULL}
};

MOD_INIT(raw_extension) {
    PyObject* m;
    MOD_DEF(m, "raw_extension", "", methods)
    return MOD_SUCCESS_VAL(m);
}

INF:cppimport.magic:C/C++ objects: raw_extension.add


In [12]:
assert 7 == raw_extension.add(2, 5)

### Use Boost.Python
To use [Boost.Python](https://www.boost.org/doc/libs/release/libs/python/), you must explicitly define libraries.


In [13]:
%%cppimport operators.cpp
// Source:
// https://github.com/TNG/boost-python-examples/blob/main/07-Operators/operators.cpp
#include <sstream>
#include <string>

class NumberLike {
public:
    NumberLike(int n = 0) : mN(n) {}
    NumberLike& operator+= (int i) {
        mN += i;
        return *this;
    }
    std::string str() const {
        std::stringstream s;
        s << mN;
        return s.str();
    }
    std::string repr() const {
        std::stringstream s;
        s << "NumberLike("<< mN << ")";
        return s.str();
    }
private:
    int mN;
};

NumberLike operator+(NumberLike n, int i) {    n += i;
    return n;
};

#include <boost/python.hpp>
using namespace boost::python;

BOOST_PYTHON_MODULE(operators) {
    class_<NumberLike>("NumberLike")
        .def(init< optional<int> >())
        .def(self + int())
        .def("__str__", &NumberLike::str)
        .def("__repr__", &NumberLike::repr)
    ;
}
/*<%
import os
import sys
_pfx = os.environ.get('CONDA_PREFIX', '/usr')

cfg['compiler_args'] += ['-std=c++14']
cfg['libraries'] += ['boost_python' +
                     str(sys.version_info.major) + str(sys.version_info.minor)]
cfg['include_dirs'] += [ os.path.join(_pfx, 'include') ]
cfg['library_dirs'] += [ os.path.join(_pfx, 'lib') ]
if sys.platform.startswith('darwin'):
    cfg['extra_link_args'] += ['-rpath', os.path.join(_pfx, 'lib')]
%>*/

INF:cppimport.magic:C/C++ objects: operators.NumberLike


In [14]:
n = operators.NumberLike(7)
m = n + 2
assert str(m) == "9"
n0 = operators.NumberLike()
m0 = n0 + 1
assert repr(m0) == "NumberLike(1)"
print(n, m, str(m), n0, m0, repr(m0))

7 9 9 0 1 NumberLike(1)


## Use extra source
Since the module is built in a temporary directory, references to additional source files must be absolute.

In [15]:
%%writefile .temp_test_extra_sources1.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources1.cpp
int square(int x) {
    return x * x;
}

Overwriting .temp_test_extra_sources1.cpp


In [16]:
%%cppimport extra_sources.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/extra_sources.cpp
#include <pybind11/pybind11.h>

int square(int x);

int square_sum(int x, int y) {
    return square(x) + square(y);
}

PYBIND11_MODULE(extra_sources, m) {
    m.def("square_sum", &square_sum);
}
/*<%
import os
setup_pybind11(cfg)
cfg['sources'] = [os.path.abspath('.temp_test_extra_sources1.cpp')]
cfg['parallel'] = True
%>*/

INF:cppimport.magic:C/C++ objects: extra_sources.square_sum


In [17]:
assert 25 == extra_sources.square_sum(3, 4)

<a id='use_header_files'></a>
## Use header files

Since the module is being built in a temporary directory, references to additional include directories, including current directory (`.`), must be absolute.

In [18]:
%%writefile .temp_test_thing.h
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/test_cppimport.py
#include <iostream>
struct Thing {
    void cheer() {
        std::cout << "WAHHOOOO" << std::endl;
    }
};
#define THING_DEFINED

Overwriting .temp_test_thing.h


In [19]:
%%writefile .temp_test_thing2.h
// This file is intentionally left empty.

Overwriting .temp_test_thing2.h


In [20]:
%%cppimport mymodule.cpp
// Source:
// https://github.com/tbenthompson/cppimport/blob/main/tests/mymodule.cpp
#include <pybind11/pybind11.h>
#include ".temp_test_thing.h"
#include ".temp_test_thing2.h"

namespace py = pybind11;

int add(int i, int j) {
    return i + j;
}

PYBIND11_MODULE(mymodule, m) {
    m.def("add", &add);
#ifdef THING_DEFINED
    // #pragma message "stuff"
    py::class_<Thing>(m, "Thing", py::module_local())
        .def(py::init<>())
        .def("cheer", &Thing::cheer);
#endif
}
/*<%
import os
setup_pybind11(cfg)
cfg['include_dirs'] += [os.path.abspath('.')]
cfg['dependencies'] += [os.path.abspath('.temp_test_thing.h')]
%>*/

INF:cppimport.magic:C/C++ objects: mymodule.add mymodule.Thing


In [21]:
assert 8 == mymodule.add(3, 5)

In [22]:
mymodule.Thing().cheer()  # pytest: skip: `std::cout<<...` don't captured

WAHHOOOO


## Numpy, OpenMP, BLAS

`macOS` `Xcode` `clang` don't have `OpenMP`, we need `clang`  in `PATH` form `MacPorts`, `Homebrew` or something else.

In [23]:
import os
import re
import sys

if sys.platform.startswith("darwin"):
    for p in ["/opt/local/bin", "/opt/homebrew/bin", "/usr/local/opt/llvm@15/bin"]:
        if not re.search(p + ".*:/usr/bin", os.environ["PATH"]):
            os.environ["PATH"] = p + ":" + os.environ["PATH"]

In [24]:
%%cppimport pnob.cpp
#include <algorithm>
#include <string>
#include <unordered_map>

#include <cblas.h>
#include <omp.h>
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

template<class T>
T dot(const T *pa, const T *pb, size_t n) {
    T s = 0.;
    while(n-- > 0) {
        s += (*pa++)*(*pb++);
    }
    return s;
}

template<class T>
void gemm(CBLAS_ORDER _order, CBLAS_TRANSPOSE _transa, CBLAS_TRANSPOSE _transb,
          py::ssize_t m, py::ssize_t n, py::ssize_t k, 
          T _alpha, 
          const T *pa, py::ssize_t lda, const T *pb, py::ssize_t ldb, 
          T _beta, 
          T *pc, py::ssize_t ldc) {
    if(_order != CblasRowMajor || 
       _transa != CblasNoTrans || _transb != CblasTrans ||
       _alpha != 1. || _beta != 0.
      ) {
        throw py::value_error(std::string(__func__) + 
                              ": Bad unimplemented operands");
    }

    const size_t L2cache = 256*1024;
    const py::ssize_t grid_j = std::max((size_t)1, 
                                  L2cache/sizeof(T)/(3*k));
    const py::ssize_t grid_i = std::max((size_t)1, 
                                  (L2cache/sizeof(T) - k*grid_j)/(2*k));

    #pragma omp parallel for
    for(py::ssize_t ig = 0; ig < m; ig += grid_i) {
        for(py::ssize_t jg = 0; jg < n; jg += grid_j) {
            if(PyErr_CheckSignals()) {
                // Throw an exception in a parallel loop can cause a crash, 
                // so we break the loop
                break; 
            }
            for(py::ssize_t i = ig; i < std::min(ig + grid_i, m); i++) {
                for(py::ssize_t j = jg; j < std::min(jg + grid_j, n); j++) {
                    pc[i*ldc + j] = dot(pa + i*lda, pb + j*ldb, k);
                }
            }
        }
    }
    if(PyErr_CheckSignals()) {
        throw py::error_already_set();
    }
}

template<> 
void gemm(CBLAS_ORDER order, CBLAS_TRANSPOSE transa, CBLAS_TRANSPOSE transb,
          py::ssize_t m, py::ssize_t n, py::ssize_t k,
          double alpha,
          const double *pa, py::ssize_t lda, const double *pb, py::ssize_t ldb,
          double beta,
          double *pc, py::ssize_t ldc) {
    cblas_dgemm(order, transa, transb,
                m, n, k,
                alpha, pa, lda, pb, ldb, beta,
                pc, ldc);
}

template<class T>
py::array_t<T> matmulNT(py::array_t<T, py::array::c_style> _a, 
                        py::array_t<T, py::array::c_style> _b) {
    auto a = _a.template unchecked<2>();
    auto b = _b.template unchecked<2>();

    const auto m = a.shape(0);
    const auto k = a.shape(1);
    const auto n = b.shape(0);

    if(k != b.shape(1)) {
        throw std::invalid_argument(
            std::string(__func__) + 
            ": Operands dimensions mismatch: a.shape(1) != b.shape(1)"
        );
    }
    
    py::array_t<T> _r({m, n});
    auto r = _r.template mutable_unchecked<2>();

    gemm(CblasRowMajor, CblasNoTrans, CblasTrans,
         m, n, k,
         T(1.), &a(0, 0), k, &b(0, 0), k, T(0.),
         &r(0,  0), n);
    return _r;
}

std::string omp_version() {
    std::unordered_map<unsigned,std::string> map { 
        { 199710, "1.0 Fortran" },
        { 199810, "1.0 C/C++" },
        { 199911, "1.1 Fortran" },
        { 200011, "2.0 Fortran" },
        { 200203, "2.0 C/C++" },
        { 200505, "2.5 C/C++/Fortran" }, 
        { 200805, "3.0 C/C++/Fortran" },
        { 201107, "3.1 C/C++/Fortran" },
        { 201307, "4.0 C/C++/Fortran" },
        { 201511, "4.5 C/C++/Fortran" },
        { 201811, "5.0 C/C++/Fortran" },
        { 202011, "5.1 C/C++/Fortran" },
        { 202111, "5.2 C/C++/Fortran" }
    };
    auto pi = map.find(_OPENMP);
    return (pi != map.end() 
            ? pi->second
            : std::to_string(_OPENMP));
}

PYBIND11_MODULE(pnob, m) {
    m.def("matmulNT", &matmulNT<float>);
    m.def("matmulNT", &matmulNT<double>, py::prepend{});
    m.def("matmulNT", &matmulNT<long double>, py::prepend{});
    m.def("omp_version", omp_version);
}
/*<%
setup_pybind11(cfg)
cfg['compiler_args'] += ['-fopenmp', '-std=c++17', '-march=native', '-O3']
cfg['libraries'] += ['omp', 'cblas']
%>*/

INF:cppimport.magic:C/C++ objects: pnob.matmulNT pnob.omp_version


In [25]:
import numpy as np
import threadpoolctl

print(pnob.omp_version())

5.0 C/C++/Fortran


In [26]:
import timeit


def ptimeit(limit, stmt):
    cnt, tcnt = timeit.Timer(stmt=stmt, globals=globals()).autorange()
    print("%s: %-24s : %g" % (limit, stmt, tcnt / cnt))


m, n = 540, 360
for t in [np.float32, np.float64, np.float128]:
    a = t(np.random.uniform(size=(m, n)))
    b = t(np.random.uniform(size=(m, n)))
    print(t)
    c = a @ b.T
    c1 = pnob.matmulNT(a, b)
    np.testing.assert_allclose(c1, c, rtol=2 * np.sqrt(n) * np.finfo(t).eps)
    with threadpoolctl.threadpool_limits(limits=1):
        ptimeit("seq", "c = a @ b.T")
        ptimeit("seq", "c1 = pnob.matmulNT(a, b)")
    ptimeit("", "c = a @ b.T")
    ptimeit("", "c1 = pnob.matmulNT(a, b)")

<class 'numpy.float32'>
seq: c = a @ b.T              : 0.00202156
seq: c1 = pnob.matmulNT(a, b) : 0.0910553
: c = a @ b.T              : 0.000632908
: c1 = pnob.matmulNT(a, b) : 0.0129897
<class 'numpy.float64'>
seq: c = a @ b.T              : 0.00444282
seq: c1 = pnob.matmulNT(a, b) : 0.00471878
: c = a @ b.T              : 0.0020692
: c1 = pnob.matmulNT(a, b) : 0.00213252
<class 'numpy.longdouble'>
seq: c = a @ b.T              : 0.361845
seq: c1 = pnob.matmulNT(a, b) : 0.109525
: c = a @ b.T              : 0.318189
: c1 = pnob.matmulNT(a, b) : 0.0206091


## Save options
Verbosity options of `%cppimport` overrides saved default verbosity options.

We can see whatever the default config has:

In [27]:
%cppimport_config

INF:cppimport.magic:Current defaults arguments for %%cppimport: -v


To clear the custom defaults and back to the defaults (no arguments) use:

In [28]:
%cppimport_config --defaults

INF:cppimport.magic:Deleted custom config. Back to default arguments for %%cppimport


## cppimport configuration
### `cfg`
Basically, selected subset of arguments of `class distutils.core.Extension`:

Field                | Type | Description
:--------------------|:-----|:-----------
"dependencies"       | str  | Additional dependencies (e.g. header files!)
"extra_compile_args" | str  | Any extra platform- and compiler-specific information to use when compiling the source files in ‘sources’
"extra_link_args"    | str  | Any extra platform- and compiler-specific information to use when linking object files together to create the extension (or to create a new static Python interpreter)
"include_dirs"       | str  | List of directories to search for C/C++ header files (in Unix form for portability)
"libraries"          | str  | List of directories to search for C/C++ libraries at link time
"library_dirs"       | str  | List of directories to search for C/C++ libraries at link time
"parallel"           | bool | Enable parallel compilation
"sources"            | str  | List of source filenames, relative to the distribution root (where the setup script lives), in Unix form (slash-separated) for portability. Source files may be C, C++, SWIG (.i), platform-specific resource files, or whatever else is recognized by the build_ext command as source for a Python extension.

### `cppimport.settings`

Field                      | Type      | Description
:--------------------------|:----------|:-----------
"force_rebuild"            | bool      | Build and load cell as if pseudorandom unique checksum (`cppimport`: Rebuild even when the checksum matches)
"file_exts"                | list[str] | List recoginzed source file extensions
"lock_suffix"              | str       | Suffix of lock file
"lock_timeout"             | int       | Lock timeout [s]
"release_mode"             | bool      | Skip the checksum and compiled binary existence checks during importing
"remove_strict_prototypes" | bool      | Remove flag `-Wstrict-prototypes` from `distutils.sysconfig`
"rtld_flags"               | int       | `sys.setdlopenflags()` flags, `cppimport` default `os.RTLD_LOCAL`


## Internal `cppimport.magic`
Build and reloading module of the modified cell occurs as follows:
1. Calculating checksum;
2. Checking already loaded module, if exist returning it (macOS/Linux/FreeBSD - ignoring dynamic symbol redefinition, Windows - simple locks loaded extension DLL, we can't overwrite it);
3. Creating cache directory, writing cell, building and loading.


In [29]:
import sys

(mymodule.__file__, [k for k in sys.modules if "mymodule" in k], mymodule.__name__)

('/Users/leo/.cache/ipython/cppimport/e291ddc0/116a5fed89d919a3192ba1938bf95415/_116a5fed89d919a3192ba1938bf95415_mymodule.cpython-311-darwin.so',
 ['_116a5fed89d919a3192ba1938bf95415_mymodule'],
 'mymodule')

## Unimplemented

Now unimplemented cell's C/C++ syntax highlighting for `Jupyter Lab` (now works only for `Jupyter Nootebook`).

## Cell tags
This notebook use next tags:

Tags          | Descriptions
:-------------|:-------------
`random`      | Tests don't check outpus tagged cells.
`random_long` | `Clear Outputs` before commit.
`skip`, `skip_darwin`, `skip_linux`, `skip_win32`, `skip_python_3_12`, ... `skip_python_le3_09`, ... | Skip cell (tests don't compute)
`xfail`, `xfail_darwin`, `xfail_linux`, `xfail_win32`, `xfail_python_3_12`, ... `xfail_python_le3_09`, ... | Cell compute, expected fail.

## Additional test

For test purpose we replay cells above with and without `cppimport.settings['force_rebuild'] = False`.

In [30]:
last_replay_test = len(In)

In [ ]:
import re
import sys

import cppimport

force_assert_fail = -1 # 4
#
# WARNING: `cppimport_config -v` for test `force_rebuild`
#
for test_case in [("cppimport.settings['force_rebuild'] = False; "
                   "get_ipython().run_line_magic('cppimport_config', '-v'); "
                  ),
                  ("cppimport.settings['force_rebuild'] = True; "
                   "get_ipython().run_line_magic('cppimport_config', '--defaults'); "
                  ),
                  #("get_ipython().run_line_magic('cppimport_config', '-vvv');"
                  #)
                 ]:
    print('\033[95m exec( ',
      test_case,
      ' )\033[0m', flush=True)
    exec(test_case)
    for i in range(first_replay_test, last_replay_test-1):
        ii = In[i]
        if force_assert_fail != 0:
            force_assert_fail -= 1
        elif 'assert' in ii:
            ii = re.sub(r'(assert)\s\s*([0-9])', r'\1 1\2', ii)
            ii = re.sub(r'(assert)\s\s*([^0-9])', r'\1 " " + \2', ii)
        print('\033[92m exec(\033[0m',
              ii.replace('\\n', '\n').replace("\\'", "'"),
              '\033[92m )\033[0m', flush=True)
        exec(ii)
        sys.stdout.flush()
        sys.stderr.flush()

In [32]:
# WARNING: Restore default configuration
#
%cppimport_config --defaults
cppimport.settings["force_rebuild"] = False

In [33]:
print("Successful magic doc test, 2+2 =", 2 + 2)

Successful magic doc test, 2+2 = 4
